**Лабораторная работа №1** Вариант 9

**Цель лабораторной работы:** познакомиться с применением модели машинного обучения для решения задачи классификации (метод логистической регрессии).

Проанализируем известный датасет cafe. Датасет содержит сведения о заказах в кафе. Также он содержит сведения о сумме заказа, наименованиях купленных товаров. Смысл задачи - определять основываясь на этих данных, был ли в заказе напиток. Набор данных включает следующие атрибуты:

* order_id – номер заказа;
* quantity – количество;
* item_name – имя элемента;
* choice_description – описание элемента;
* item_price – цена элемента;

**1. Импортируем библиотеки и загружаем данные**:

In [499]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

In [ ]:
df = pd.read_csv('/cafe.csv')
df.head()

**2. Выведем целевую переменную, целевые имена и частоту каждого уникального значения:**

In [ ]:
df['item_price_numeric'] = df['item_price'].str.replace('$', '').str.replace(',', '.').astype(float)

def is_drink(item_name):
    drinks = ['Nantucket Nectar', 'Canned Soda', 'Izze', 'Bottled Water', '6 Pack Soft Drink', 'Canned Soft Drink']
    return 1 if any(drink in str(item_name) for drink in drinks) else 0

df['is_drink'] = df['item_name'].apply(is_drink)


order_analysis = df.groupby('order_id').apply(lambda x: pd.Series({
    'total_items': x['quantity'].sum(),
    'food_items': (x['quantity'] * (1 - x['is_drink'])).sum(),
    'drink_items': (x['quantity'] * x['is_drink']).sum(),
    'total_price': x['item_price_numeric'].sum(),
    'unique_items': len(x)
})).reset_index()
order_analysis = order_analysis[order_analysis['food_items'] > 0]
order_analysis['ordered_drink'] = (order_analysis['drink_items'] > 0).astype(int)

(unique, counts) = np.unique(order_analysis['ordered_drink'], return_counts=True)
print("Unique values of the target variable: ", unique)
print("Counts of the target variable: ", counts)
order_analysis.head(60)

**3. Построим столбчатую диаграмму, чтобы увидеть целевую переменную:**

In [ ]:
sns.set_style("whitegrid")
sns.countplot(order_analysis, x='ordered_drink')

In [ ]:
order_analysis.info()

**4. Проверяем наличие пропущенных значений и выбросов:**

In [ ]:
print(order_analysis.isnull().sum())

In [ ]:
sns.boxplot(x=order_analysis['total_price'])

**5. Исключаем невлияющие категориальные признаки:**

In [ ]:
order_analysis_copy = order_analysis.copy()
non_influencing_signs = ['order_id', 'unique_items', 'total_items', 'total_price', 'drink_items']
order_analysis_copy.drop(non_influencing_signs, axis=1, inplace=True)
order_analysis_copy.head()

**6. Разделяем данные на признаки и целевую переменную:**

In [507]:
y = order_analysis_copy['ordered_drink']
x = order_analysis_copy.drop('ordered_drink', axis=1)

**7. Применяем операцию нормализации для численной устойчивости:**

In [508]:
standardizer = StandardScaler()
x = standardizer.fit_transform(x)

**8. Разделяем данные на тренировочный и тестовый наборы:**

In [509]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=90)

**9. Обучаем модель логистической регрессии:**

In [ ]:
model = LogisticRegression()
model.fit(x_train, y_train)

**10. Делаем прогноз на основе данных тестирования:**

In [511]:
y_pred = model.predict(x_test)

**11. Рассчитываем показатели точности, сравнив фактические и прогнозируемые значения:**

In [ ]:
cm = confusion_matrix(y_test, y_pred)
TN, FP, FN, TP = confusion_matrix(y_test, y_pred).ravel()
print("True Negative (TN) = ", TN)
print("False Positive (FP) = ", FP)
print("False Negative (FN) = ", FN)
print("True Positive (TP) = ", TP)

accuracy = (TP + TN) / (TN + TP + FP + FN)
print("Accuracy of the binary classification = {:0.3f}".format(accuracy))

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

sns.heatmap(cm, square=True, annot=True, fmt='d', cbar=False, cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность: {accuracy:.2f}")

In [ ]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print(f"Точность: {precision:.2f}")
print(f"Полнота: {recall:.2f}")

In [ ]:
f1 = f1_score(y_test, y_pred)
print(f"F1-мера: {f1:.2f}")

In [ ]:
order_analysis_copy.corr()

**12. Строим ROC-кривую:**

In [ ]:
y_score = model.predict_proba(x_test)[:, 1]
fpr, tpr,  thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classfier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

Таким образом, в лабораторной работе №3 была проанализирована зависимость наличия напитка в заказе по количеству еды в нем с помощью метода логистической регрессии.